In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt
import sys
import matplotlib.image
dtype = tf.float32

In [ ]:
train_x = np.random.rand(100, 500)
train_y = np.random.rand(100, 50)
test_x = np.random.rand(30, 500)
test_y = np.random.rand(30, 50)

In [8]:
def weight_variable(shape, name):
    # return tf.get_variable(name, shape=shape, dtype=dtype, initializer=tf.contrib.layers.xavier_initializer())
    initial = tf.truncated_normal(shape, stddev=0.1, dtype=tf.float32)
    return tf.Variable(initial, name=name, dtype=tf.float32)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape, dtype=dtype)
    return tf.Variable(initial, name=name, dtype=dtype)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv_layer(x, r, c, i, o, name):
    W_conv = weight_variable([r, c, i, o], name + '_w')
    b_conv = bias_variable([o], name + '_b')
    return tf.nn.relu(conv2d(x, W_conv) + b_conv)

def inception_layer(x, i, o11, o21, o22, o31, o32, o41, name):    # input, input_layers, output_layers 1 and 2
    conv1x1_1 = conv_layer(x, 1, 1, i, o11, name + 'conv1x1_1')
    conv1x1_2 = conv_layer(x, 1, 1, i, o21, name + 'conv1x1_2')
    conv3x3 = conv_layer(conv1x1_2, 5, 30, o21, o22, name + 'conv3x3')
    conv1x1_3 = conv_layer(x, 1, 1, i, o31, name + 'conv1x1_3')
    conv5x5 = conv_layer(conv1x1_3, 10, 30, o31, o32, name + 'conv5x5')
    max3x3 = max_pool(x, 3,3, 1)
    conv1x1_4 = conv_layer(max3x3, 1, 1, i, o41, name + 'conv1x1_4')
    return tf.nn.relu(tf.concat([conv1x1_1, conv3x3, conv5x5, conv1x1_4], 3))

def max_pool(x, w, h, s):
    return tf.nn.max_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='SAME')

def avg_pool(x, w, h, s):
    return tf.nn.avg_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='SAME')
def avg_pool_final(x, w, h, s):
    return tf.nn.avg_pool(x, ksize=[1, w, h, 1],
                          strides=[1, s, s, 1], padding='VALID')

def fully_connected(X, layers, out, name):
    pre_l = 0
    i = 1
    for l in layers:
        if not pre_l:
            pre_l = l
            continue
        w_h = weight_variable([pre_l, l], name + '_' + str(i) + '_w')
        b_h = bias_variable([l], name + '_' + str(i) + '_b')
        X = tf.nn.relu(tf.matmul(X, w_h) + b_h)
        pre_l = l
        i += 1
    w_h = weight_variable([pre_l, out], name + '_' + str(i) + '_w')
    b_h = bias_variable([out], name + '_' + str(i) + '_b')
    return tf.matmul(X, w_h) + b_h

In [26]:
#def main():
    #data loading
    print("Reading Data..")
    #train_x, train_y, test_x, test_y, check_x = read_data()
    print('loading data finished!')
    print('training set: ', train_x.shape, ', test set: ', test_x.shape, "\n\n")
    
    #Initializations
    print("Initializing variables..")
    BATCH_SIZE = 10
    learning_rate = 1.00e-4
    epoch = 0 
    epochNum = 1 
    data_size = len(train_x)
    
    #Model creation: placeholders
    print("Creating Model...")
    X = tf.placeholder(dtype, [None, 500])
    Y = tf.placeholder(dtype, [None, 50])
    tr = tf.placeholder(dtype)
    pr = tf.placeholder(dtype)
    
    #convolution
    x_ = tf.reshape(X, [-1, 50, 10, 1])     # 28 x 28 x 1
    print("x_", x_)
    conv = conv_layer(x_, 5, 10, 1, 64, 'conv1')      # 28 x 28 x 64
    print("conv1", conv)
    conv = conv_layer(conv, 3, 10, 64, 128, 'conv2')   # 28 x 28 x 128
    print("conv2", conv)
    pool = max_pool(conv, 10, 5, 2)             # 14 x 14 x 128
    print("pool", pool)
    norm = tf.contrib.layers.batch_norm(pool)
    print("norm", norm)
    conv = inception_layer(norm, 128, 8, 64, 96, 8, 16, 8, 'inc1')  # 14 x 14 x 128
    print("inc1", conv)
    conv = inception_layer(conv, 128, 64, 96, 128, 16, 32, 32, 'inc2')  # 14 x 14 x 256
    print("inc2", conv)
    conv = inception_layer(conv, 256, 160, 112, 224, 24, 64, 64, 'inc3')  # 14 x 14 x 512
    print("inc3", conv)
    pool = max_pool(conv, 7, 5, 2)             # 7 x 7 x 256
    print("pool", pool)
    norm = tf.contrib.layers.batch_norm(pool)
    print("norm", norm)
    conv = inception_layer(norm, 512, 128, 128, 256, 32, 64, 64, 'inc4')  # 7 x 7 x 512
    print("inc4", conv)
    pool = avg_pool(conv, 7, 5, 2)             # 1 x 1 x 512 || 10x7x2x512
    print("pool", pool)
    pool = avg_pool_final(pool, 7, 2, 1)             # 1 x 1 x 512 || 10x7x2x512
    print("poolfinal", pool)

    #FCL and dropout
    cnn = tf.reshape(pool, [-1, 512])
    print("cnn", cnn)
    cnn = tf.nn.dropout(cnn, pr)
    print("cnn dropout", cnn)
    out_y = fully_connected(cnn, [512], 50, 'fc1')
    print("yhat", out_y)
    print("y", Y.shape)

    
    #loss function
    loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions = out_y, labels = Y)) 

    #training 
    train_step = tf.train.AdamOptimizer(tr).minimize(loss)
    
    sess = tf.Session()
    with sess.as_default() as sess:
        tf.global_variables_initializer().run()
        print("Training...")
        while epoch < epochNum:
            batchIter = 0; 
            epoch += 1
            for start in range(0, data_size, BATCH_SIZE):
                batchIter += 1
                end = start + BATCH_SIZE
                sess.run(train_step, feed_dict={X: train_x[start:end], Y: train_y[start:end], tr: learning_rate, pr: 0.45})
                batchPer = 100*batchIter*BATCH_SIZE/data_size
                if round(batchPer) % 5 == 0:
                    print('percent of batch: ', batchPer) 
            #print accuracy 
            accTrain = sess.run(loss, feed_dict={X: train_x, Y: train_y, pr: 1.0})
            accTest = sess.run(loss, feed_dict={X: test_x, Y: test_y, pr: 1.0})
            print('epoch: ', epoch, ', Train Accuracy: ', accTrain,'Test Accuracy:', accTest, ', learning_rate: ', learning_rate)
#if __name__ == "__main__":
#    main()


Reading Data..
loading data finished!
training set:  (100, 500) , test set:  (30, 500) 


Initializing variables..
Creating Model...
x_ Tensor("Reshape_7:0", shape=(?, 50, 10, 1), dtype=float32)
conv1 Tensor("Relu_90:0", shape=(?, 50, 10, 64), dtype=float32)
conv2 Tensor("Relu_91:0", shape=(?, 50, 10, 128), dtype=float32)
pool Tensor("MaxPool_18:0", shape=(?, 25, 5, 128), dtype=float32)
norm Tensor("BatchNorm_6/FusedBatchNorm:0", shape=(?, 25, 5, 128), dtype=float32)
inc1 Tensor("Relu_98:0", shape=(?, 25, 5, 128), dtype=float32)
inc2 Tensor("Relu_105:0", shape=(?, 25, 5, 256), dtype=float32)
inc3 Tensor("Relu_112:0", shape=(?, 25, 5, 512), dtype=float32)
pool Tensor("MaxPool_22:0", shape=(?, 13, 3, 512), dtype=float32)
norm Tensor("BatchNorm_7/FusedBatchNorm:0", shape=(?, 13, 3, 512), dtype=float32)
inc4 Tensor("Relu_119:0", shape=(?, 13, 3, 512), dtype=float32)
pool Tensor("AvgPool_6:0", shape=(?, 7, 2, 512), dtype=float32)
poolfinal Tensor("AvgPool_7:0", shape=(?, 1, 1, 512), dtype=f

In [28]:
accValue = sess.run(loss, feed_dict={X: test_x, Y: test_y, pr: 1.0})
yhat = sess.run(out_y, feed_dict={X: test_x[1,:], pr: 1.0})
print(accValue)

3.7966406
